In [79]:
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import os
import math
import re


In [80]:
mp_pose = mp.solutions.pose


In [81]:
landmarks_points = {
    "left_shoulder": 0, "right_shoulder": 1,
    "left_elbow": 2, "right_elbow": 3,
    "left_wrist": 4, "right_wrist": 5,
    "left_hip": 6, "right_hip": 7,
    "left_knee": 8, "right_knee": 9,
    "left_ankle": 10, "right_ankle": 11,
    "left_heel": 12, "right_heel": 13,
    "left_foot_index": 14, "right_foot_index": 15,
}


In [83]:
def angle(pt1, pt2, pt3):
    angle_calc = 0
    a = (pt1[0], pt1[1])
    b = (pt2[0], pt2[1])
    c = (pt3[0], pt3[1])
    ang = math.degrees(math.atan2(
        c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    angle_calc = ang + 360 if ang < 0 else ang
    angle_calc = 360 - angle_calc if angle_calc > 215 else angle_calc
    return angle_calc


In [84]:
pose_list = []

file = open("./csv_files/angles.csv", "a")

main_dir = "./poses_dataset/Images"
pose_dir_list = os.listdir(main_dir)

for i in range(0, len(pose_dir_list)):
    if i in [0, 2, 6, 7, 9]:
        continue
    step_list = []
    wrong_right_dir_list = os.listdir(f"{main_dir}/{pose_dir_list[i]}")

    for j in range(0, len(wrong_right_dir_list) - 1):
        steps_dir_list = os.listdir(
            f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}")

        for k in range(0, len(steps_dir_list)):
            sl = []
            images_dir_list = os.listdir(
                f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}")
            if j == 1:
                reg = re.search(r"([0-9])", steps_dir_list[k])

            for l in range(0, len(images_dir_list)):
                pre_list = []
                with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
                    image = cv2.imread(
                        f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                    result = pose.process(
                        cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    try:
                        predict = True
                        for landmarks in result.pose_landmarks.landmark:  # change to pose_world_landmarks if needed
                            pre_list.append(landmarks)
                    except AttributeError:
                        print(
                            f"No points {main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                        predict = False

                if predict == True:
                    gen1116 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y
                        ] for m in range(11, 17)
                    ]).tolist()

                    gen2333 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y
                        ] for m in range(23, 33)
                    ]).tolist()

                    gen1116.extend(gen2333)

                    all_lm = gen1116.copy()

                    tmp = []

                    armpit_left = angle(
                        all_lm[landmarks_points["left_elbow"]],
                        all_lm[landmarks_points["left_shoulder"]],
                        all_lm[landmarks_points["left_hip"]]
                    )
                    armpit_right = angle(
                        all_lm[landmarks_points["right_elbow"]],
                        all_lm[landmarks_points["right_shoulder"]],
                        all_lm[landmarks_points["right_hip"]]
                    )

                    elbow_left = angle(
                        all_lm[landmarks_points["left_shoulder"]],
                        all_lm[landmarks_points["left_elbow"]],
                        all_lm[landmarks_points["left_wrist"]]
                    )
                    elbow_right = angle(
                        all_lm[landmarks_points["right_shoulder"]],
                        all_lm[landmarks_points["right_elbow"]],
                        all_lm[landmarks_points["right_wrist"]]
                    )

                    hip_left = angle(
                        all_lm[landmarks_points["right_hip"]],
                        all_lm[landmarks_points["left_hip"]],
                        all_lm[landmarks_points["left_knee"]]
                    )
                    hip_right = angle(
                        all_lm[landmarks_points["left_hip"]],
                        all_lm[landmarks_points["right_hip"]],
                        all_lm[landmarks_points["right_knee"]]
                    )

                    knee_left = angle(
                        all_lm[landmarks_points["left_hip"]],
                        all_lm[landmarks_points["left_knee"]],
                        all_lm[landmarks_points["left_ankle"]]
                    )
                    knee_right = angle(
                        all_lm[landmarks_points["right_hip"]],
                        all_lm[landmarks_points["right_knee"]],
                        all_lm[landmarks_points["right_ankle"]]
                    )

                    ankle_left = angle(
                        all_lm[landmarks_points["left_knee"]],
                        all_lm[landmarks_points["left_ankle"]],
                        all_lm[landmarks_points["left_foot_index"]]
                    )
                    ankle_right = angle(
                        all_lm[landmarks_points["right_knee"]],
                        all_lm[landmarks_points["right_ankle"]],
                        all_lm[landmarks_points["right_foot_index"]]
                    )

                    tmp = [
                        armpit_left,
                        armpit_right,
                        elbow_left,
                        elbow_right,
                        hip_left,
                        hip_right,
                        knee_left,
                        knee_right,
                        ankle_left,
                        ankle_right
                    ]

                    sl.append(tmp)
                    tmp_data = pd.DataFrame(sl)
                    
            file.write(f"{i},{k},{tmp_data[0].median()},{tmp_data[1].median()},{tmp_data[2].median()},{tmp_data[3].median()},{tmp_data[4].median()},{tmp_data[5].median()},{tmp_data[6].median()},{tmp_data[7].median()},{tmp_data[8].median()},{tmp_data[9].median()}\n")

file.close()


No points ./poses_dataset/Images/Ardhakati_Chakrasana/Ardhakati_Chakrasana_Right_Steps/Ardhakati_Chakrasana_Right_Step_1/20230311_083742.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_1/20230311_151821.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_1/20230311_151924.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_2/20230311_153402(0).jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_2/20230311_154237.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_2/20230311_154240.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_3/20230311_155356.jpg
No points ./poses_dataset/Images/Marjariasana/Marjariasana_Right_Steps/Marjariasana_Right_Step_3/20230311_155409(0).jpg
No points ./poses_dataset/Images/M